In [1]:
%cd ..

/home/zaccharie/workspace/fastmri-reproducible-benchmark


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
from keras.callbacks import TensorBoard
from keras.utils.vis_utils import model_to_dot
from keras_tqdm import TQDMNotebookCallback
from keras import backend as K
from tqdm import tqdm_notebook

from data import zero_filled_2d_generator
from unet import unet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# paths
train_path = '/media/zaccharie/BACKUP/singlecoil_train/'
val_path = '/home/zaccharie/workspace/fastmri-reproducible-benchmark/singlecoil_val'
test_path = '/home/zaccharie/workspace/fastmri-reproducible-benchmark/singlecoil_test'

In [ ]:
n_samples_train = 7335
n_samples_val = 800

In [5]:
# generators
train_gen = zero_filled_2d_generator(train_path)
val_gen = zero_filled_2d_generator(val_path)

In [6]:
run_params = {'n_layers': 4, 'pool': 'average', "layers_n_channels": [1, 2, 4, 8]}
n_epochs = 2

In [ ]:
model = unet(input_size=(320, 320, 1), with_extra_sigmoid=True, **run_params)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    batch_size=32, 
    write_graph=True, 
    write_images=True, 
)
print(model.summary())
model.fit_generator(
    train_gen, 
    steps_per_epoch=int(n_samples_train / batch_size), 
    epochs=n_epochs,
    validation_data=val_gen,
    validation_steps=int(n_samples_val / batch_size),
    verbose=1,
    callbacks=[TQDMNotebookCallback(), tboard_cback],
)
# K.clear_session() 